In [1]:
import subprocess

In [2]:
CODE_PANIIKKI_UPTIME='''
from concurrent.futures import ThreadPoolExecutor
import subprocess

paniikki = [
    "befunge","bit","bogo","brainfuck","deadfish","emo","entropy","false","fractran","fugue","glass","haifu","headache","intercal","malbolge","numberwang","ook","piet","regexpl","remorse","rename","shakespeare","smith","smurf","spaghetti","thue","unlambda","wake","whenever","whitespace","zombie"
]
def get_luokka(nimi):
    try:
        return nimi, subprocess.check_output(["ssh", nimi, "uptime"], timeout=1, stderr=subprocess.STDOUT).decode("utf-8").rstrip("\\n")
    except subprocess.TimeoutExpired:
        return nimi, "computer doesn't answer"
    except subprocess.CalledProcessError as e:
        return nimi, "error: %s" % e.output.decode("utf-8").rstrip("\\n")

executor = ThreadPoolExecutor(max_workers=8)

for nimi, tulos in executor.map(get_luokka, paniikki):
    print("%s: %s" % (nimi, tulos))
'''

In [ ]:
subprocess.check_output(["cat check_paniikki_uptime.py"], timeout=1, 
                        stderr=subprocess.STDOUT).decode("utf-8").rstrip("\n")

In [11]:
with open('temp.py', 'w') as f:
    f.write(CODE_PANIIKKI_UPTIME)
    
uptimes = subprocess.check_output(["ssh kosh python3 < ./temp.py"], timeout=10, shell=True, stderr=subprocess.STDOUT).decode("utf-8").rstrip("\n")
uptimes


"befunge:  14:22:24 up 9 days, 12:30,  1 user,  load average: 0.00, 0.00, 0.00\nbit:  14:22:24 up 55 min,  1 user,  load average: 0.00, 0.00, 0.06\nbogo:  14:22:24 up 9 days, 12:30,  3 users,  load average: 0.03, 0.02, 0.00\nbrainfuck:  14:22:25 up 3 days,  3:06,  1 user,  load average: 0.00, 0.02, 0.00\ndeadfish:  14:22:25 up 9 days, 12:30,  0 users,  load average: 0.04, 0.01, 0.00\nemo:  14:22:25 up 4 days, 18:53,  0 users,  load average: 0.04, 0.03, 0.00\nentropy:  14:22:25 up 4 days,  4:22,  1 user,  load average: 1.07, 1.78, 1.60\nfalse:  14:22:25 up 52 min,  1 user,  load average: 0.53, 0.30, 0.25\nfractran: computer doesn't answer\nfugue:  14:22:25 up 2 days, 23:03,  0 users,  load average: 0.01, 0.05, 0.01\nglass:  14:22:25 up 55 min,  1 user,  load average: 0.35, 0.66, 0.69\nhaifu: computer doesn't answer\nheadache:  14:22:26 up  1:25,  0 users,  load average: 0.00, 0.00, 0.00\nintercal:  14:22:26 up 9 days, 12:30,  0 users,  load average: 0.08, 0.02, 0.01\nmalbolge:  14:22:26

In [21]:
class Node:
    '''
    'befunge:  10:35:49 up 9 days,  8:44,  0 users,  load average: 0.02, 0.01, 0.00'
    '''
    def __init__(self, uptime_output):
        self.init_properties(uptime_output)
#         self.hostname = hostname
#         self.uptime = uptime
#         self.num_users = num_users
        
    def init_properties(self, output):
        self.hostname = output.split(':')[0]
        self.uptime = ' '.join(output.split(' up ')[1].split(" user")[0].split(', ')[:-1])
        self.num_users = int(output.split(' user')[0].split()[-1])
        self.loads = output.split('average:')[-1]

    def __repr__(self):
        return repr("{:<13}=> up {:<14} with{:>2} users.  LOAD:{:>15}".format(self.hostname, self.uptime, self.num_users, self.loads))
    
n = Node('befunge:  10:35:49 up 9 days,  8:44,  0 users,  load average: 0.02, 0.01, 0.00')
print(n.__repr__())

'befunge      => up 9 days  8:44   with 0 users.  LOAD: 0.02, 0.01, 0.00'


In [19]:
class ComputerLab:
    
    def __init__(self, uptime_outputs):
        self.nodes = [Node(u) for u in uptime_outputs]
        self.sort()
        
    def sort(self):
        # Sort by number of users
        self.nodes.sort(key=lambda n: n.num_users)
        # Now sort by loads
        nodes_by_num_user_dict = {}
        for n in self.nodes:
            nodes_by_num_user_dict.setdefault(n.num_users, [])
            nodes_by_num_user_dict[n.num_users].append(n)
        for k, v in nodes_by_num_user_dict.items():
            v.sort(
                key=lambda node: 
                float(
                    node.loads.split(',')[0]
                )
            )
        # Flatten
        # https://stackoverflow.com/a/952952/3067013        
        self.nodes = [
            n for nodes in nodes_by_num_user_dict.values() 
            for n in nodes
        ]
        
            
    def __print__(self):
        title="Paniikki Nodes Status"
        print("="*len(title))
        print(title)
        print("="*len(title))
        for n in self.nodes:
            print(n)
    

#     def __repr__(self):
#         r = '\n'.join([n.__repr__() for n in nodes])
#         return repr(r)
    
# paniikki = ComputerLab(uptimes)
# paniikki.__print__()

In [28]:
class ComputerLabRemoteController:
    def __init__(self,username):
        self.username = username
        self.uptimes = None
        self.node = None
        self.port = None
        
    def check_lab_uptimes(self):
        with open('temp.py', 'w') as f:
            f.write(CODE_PANIIKKI_UPTIME)
    
        self.uptimes = subprocess.check_output(
            ["ssh kosh python3 < ./temp.py"], 
            timeout=10, 
            shell=True, # FIXME: Remove this for security
            stderr=subprocess.STDOUT
        ).decode("utf-8").rstrip("\n").split('\n')
        
        self.uptimes = [u for u in self.uptimes if "computer doesn't answer" not in u]
        
        subprocess.check_output(
            ["rm temp.py"],
            timeout=10,
            shell=True,
            stderr=subprocess.STDOUT
        ).decode("utf-8").rstrip("\n")
        
        self.lab = ComputerLab(self.uptimes)
        self.lab.__print__()
       


    def print_command(self):
        command ='''ssh {user}@{node} -t -L {port}:localhost:{port} -o ProxyCommand='ssh {user}@kosh -W %h:%p'  "bash -l -c \'module load courses/CS-E4820 ; jupyter notebook --port={port} --no-browser\'"'''.format(user=self.username,node=self.node.hostname,port         =self.port)
        print("\n")
        print("Copy/paste this line into your shell in order to start a Jupyter Notebook:\n>>>\n")
        print(command)
        print("\n")

        
    def get_port(self):
        node = self.lab.nodes[0]
        with open('temp.py', 'w') as f:
            f.write(CODE_GET_PORT.format(user=self.username,node='bit'))

        self.port = subprocess.check_output(
            ["ssh kosh python3 < ./temp.py"], 
            timeout=10, 
            shell=True, 
            stderr=subprocess.STDOUT
        ).decode("utf-8").rstrip("\n")
        
        subprocess.check_output(
            ["rm temp.py"],
            timeout=10,
            shell=True,
            stderr=subprocess.STDOUT
        ).decode("utf-8").rstrip("\n")
        
        # This is called from this function because of synchronousity
        self.print_command()
        
    def run(self):
        self.uptimes = self.check_lab_uptimes()
        self.node = self.lab.nodes[0]
        self.port = self.get_port()
        
        
c = ComputerLabRemoteController("parks1")
c.run()

Paniikki Nodes Status
'ook          => up 3 days  3:21   with 0 users.  LOAD: 0.00, 0.00, 0.00'
'numberwang   => up 9 days 12:41   with 0 users.  LOAD: 0.01, 0.02, 0.00'
'emo          => up 4 days 19:04   with 0 users.  LOAD: 0.02, 0.04, 0.00'
'intercal     => up 9 days 12:41   with 0 users.  LOAD: 0.02, 0.01, 0.00'
'wake         => up  5:43          with 0 users.  LOAD: 0.03, 0.02, 0.00'
'remorse      => up 4 days  4:11   with 0 users.  LOAD: 0.05, 0.05, 0.01'
'headache     => up  1:36          with 0 users.  LOAD: 0.07, 0.06, 0.02'
'fugue        => up 2 days 23:15   with 0 users.  LOAD: 0.09, 0.04, 0.01'
'unlambda     => up 9 days 12:41   with 0 users.  LOAD: 0.35, 0.47, 0.57'
'befunge      => up 9 days 12:42   with 1 users.  LOAD: 0.00, 0.00, 0.00'
'bit          => up  1:06          with 1 users.  LOAD: 0.00, 0.01, 0.02'
'smith        => up 3 days 18:45   with 1 users.  LOAD: 0.00, 0.00, 0.00'
'whenever     => up 9 days 12:41   with 1 users.  LOAD: 0.00, 0.00, 0.00'
'thue         =>

In [56]:
def main():
    logo='''   _         _ _            ___  __        _____  _____ 
  /_\   __ _| | |_ ___     / __\/ _\       \_   \/__   \
 //_\\ / _` | | __/ _ \   / /   \ \ _____   / /\/  / /\/
/  _  \ (_| | | || (_) | / /___ _\ \_____/\/ /_   / /   
\_/ \_/\__,_|_|\__\___/  \____/ \__/     \____/   \/    
                                                        '''
    print(logo)
    
    
    username = input("Hi! Welcome to Aalto CS-IT's Paniikki computer lab Jupyter Notebook controller. Do you want to launch one? Type in your Aalto username :D\n    e.g. tillip1\n>>> ")
    print("Thanks! Hold on a sec. This will take a few secs.")
    c = ComputerLabRemoteController(username)
    c.run()
    

if __name__ == "__main__":
    main()

   _         _ _            ___  __        _____  _____ 
  /_\   __ _| | |_ ___     / __\/ _\       \_   \/__    //_\ / _` | | __/ _ \   / /   \ \ _____   / /\/  / /\/
/  _  \ (_| | | || (_) | / /___ _\ \_____/\/ /_   / /   
\_/ \_/\__,_|_|\__\___/  \____/ \__/     \____/   \/    
                                                        
Hi! Welcome to Aalto CS-IT's Paniikki computer lab Jupyter Notebook controller. Do you want to launch one? Type in your Aalto username :D
    e.g. tillip1
>>> parks1
Thanks! Hold on a sec. This will take a few secs.


IndexError: list index out of range

In [6]:
CODE_GET_PORT='''
#!/usr/bin/python3

def is_open(ip,port):
    import socket
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    try:
        s.connect((ip, int(port)))
        s.shutdown(2)
        return True
    except:
        return False

def get_available_socket(ip,port=12520):
    while is_open(ip,port):
        port += 1
    return port
        
print(get_available_socket("{user}@{node}.org.aalto.fi"))
'''

In [ ]:
# subprocess.check_output(
#             ['''ssh parks1@smith -t -L 10321:localhost:10321 -o ProxyCommand='ssh parks1@kosh -W %h:%p'  "bash -l -c 'module load courses/CS-E4820 ; jupyter notebook --port=10321 --no-browser'"'''], 
#             timeout=10, 
#             shell=True, 
#             stderr=subprocess.STDOUT
#         ).decode("utf-8").rstrip("\n")
# 1

In [7]:
import os
import subprocess
os.path.realpath(__file__)

env = {'SSH_ASKPASS':'/', 'DISPLAY':':9999'}
p = subprocess.Popen(['ssh', '-T', '-v', 'user@myhost.com'],
    stdin=subprocess.PIPE,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    env=env,
    preexec_fn=os.setsid
)

proc = subprocess.Popen('''ssh parks1@smith -t -L 10321:localhost:10321 -o ProxyCommand='ssh parks1@kosh -W %h:%p'  "bash -l -c 'module load courses/CS-E4820 ; jupyter notebook --port=10321 --no-browser'"''',
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE)
(stdoutdata, stderrdata) = proc.communicate()

NameError: name '__file__' is not defined

In [ ]:
with open('temp.py', 'w') as f:
    f.write(CODE_GET_PORT.format(user='parks1',node='bit'))

port = subprocess.check_output(
    ["ssh kosh python3 < ./temp.py"], 
    timeout=10, 
    shell=True, 
    stderr=subprocess.STDOUT
).decode("utf-8").rstrip("\n")

subprocess.check_output(
    ["rm temp.py"],
    timeout=10,
    shell=True,
    stderr=subprocess.STDOUT
).decode("utf-8").rstrip("\n")

port

In [ ]:
import socket
def is_open(ip,port):
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    try:
        s.connect((ip, int(port)))
        s.shutdown(2)
        return True
    except:
        return False
    
# is_open("127.0.0.1", 8888)
def find_available_socket(ip,port=12520):
    while is_open(ip,port):
        port += 1
        
    return port

find_available_socket("127.0.0.1",port=8888)

In [34]:
import sys

sys.stderr.write("error")

error

In [35]:
import socket

(s1, s2) = socket.socketpair()


def setup():
    s2.close()

p = subprocess.Popen(["sudo"], stdout=s1, preexec_fn=setup)

In [36]:
p.poll()

1

In [38]:
p = subprocess.Popen(["sudo", "-S", "whoami"], stdin=subprocess.PIPE, stdout=subprocess.PIPE)
print(p.communicate("password\n"))

TypeError: memoryview: a bytes-like object is required, not 'str'

In [40]:
from subprocess import Popen, PIPE

process = Popen(['ls', '-la'], stdout=PIPE, stderr=PIPE)
stdout, stderr = process.communicate()

In [41]:
stdout

b'total 152\ndrwxr-xr-x  11 seyoung  staff    374 Jan 22 15:40 .\ndrwxr-xr-x  11 seyoung  staff    374 Dec  7 15:27 ..\n-rw-r--r--   1 seyoung  staff  12288 Jan 22 10:32 .check_paniikki_uptime.py.swp\ndrwxr-xr-x   5 seyoung  staff    170 Jan 22 10:13 .ipynb_checkpoints\n-rw-r--r--   1 seyoung  staff  27359 Jan 22 15:40 Test.ipynb\n-rw-r--r--   1 seyoung  staff   9017 Dec  9 22:12 Untitled.ipynb\n-rw-r--r--   1 seyoung  staff   4916 Dec 18 11:59 Untitled1.ipynb\n-rwxr-xr-x@  1 seyoung  staff   1546 Dec  7 14:34 check_maari_uptime.py\n-rw-r--r--   1 seyoung  staff   1058 Dec  7 15:23 check_paniikki_uptime.py\n-rw-r--r--   1 seyoung  staff     16 Dec  9 19:59 model_name\n-rw-r--r--   1 seyoung  staff   1058 Dec  9 19:06 run_ML.py\n'

In [43]:
p = subprocess.Popen(["ssh", "hello world"], stdout=subprocess.PIPE)

p.communicate()

(b'hello world\n', None)

In [46]:
import os
def run(cmd,*args):
    pid, fd = os.forkpty()
    if pid==0: # child
        os.execlp(cmd,*args)
    while True:
        data = os.read(fd,1024)
        print(data)
        if "password:" in data:    # ssh prompt
            os.write(fd,"mypassword\n")
        elif data.endswith("$ "):  # bash prompt for input
            os.write(fd,"echo hello\n")
            os.write(fd,"echo bye\n")
            os.write(fd,"exit\n")

run("ssh", "ssh", "kosh")

b"parks1@kosh.aalto.fi's password: "


TypeError: a bytes-like object is required, not 'str'

In [53]:
# pyscript = r"""
#             import sys, os;
#             verbosity=%d;
#             sys.stdin = os.fdopen(0, "rb");
#             exec(compile(sys.stdin.read(%d), "assembler.py", "exec"))
#             """ % (helpers.verbose or 0, len(content))
# pyscript = re.sub(r'\s+', ' ', pyscript.strip())

# if not rhost:
#     # ignore the --python argument when running locally; we already know
#     # which python version works.
#     argv = [sys.executable, '-c', pyscript]
# else:
#     if ssh_cmd:
#         sshl = shlex.split(ssh_cmd)
#     else:
#         sshl = ['ssh']
#     if python:
#         pycmd = "'%s' -c '%s'" % (python, pyscript)
#     else:
#         pycmd = ("P=python3; $P -V 2>/dev/null || P=python; "
#                  "exec \"$P\" -c %s") % quote(pyscript)
#         pycmd = ("exec /bin/sh -c %s" % quote(pycmd))
#     argv = (sshl +
#             portl +
#             [rhost, '--', pycmd])


argv = (["ssh"] +
        ["parks1@kosh.org.aalto.fi"])


(s1, s2) = socket.socketpair()

def setup():
    # runs in the child process
    s2.close()
s1a, s1b = os.dup(s1.fileno()), os.dup(s1.fileno())
s1.close()
p = subprocess.Popen(argv, stdin=s1a, stdout=s1b, preexec_fn=setup,
                      close_fds=True)
os.close(s1a)
os.close(s1b)
s2.sendall("content".encode("UTF8"))
# s2.sendall(content2)


In [58]:
pyscript = r"""
                import sys, os;
                verbosity=%d;
                sys.stdin = os.fdopen(0, "rb");
                exec(compile(sys.stdin.read(%d), "assembler.py", "exec"))
                """ % (0,0)

In [60]:
import re
re.sub(r'\s+', ' ', pyscript.strip())

'import sys, os; verbosity=0; sys.stdin = os.fdopen(0, "rb"); exec(compile(sys.stdin.read(0), "assembler.py", "exec"))'